# 04: OWASP Agentic Risk Detection Mapping

Maps anomaly detection capabilities to OWASP Top 10 for Agentic Applications (ASI01-ASI10).
Uses synthetic anomaly injection to create labelled evaluation data.

In [ ]:
import sys
sys.path.insert(0, '..')

import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.data.trail_loader import load_trail_dataset
from src.data.synthetic_generator import generate_anomalous_traces
from src.features.agent_extractor import AgentTraceFeatureExtractor
from src.features.ubfs_schema import UBFSNormalizer
from src.models.isolation_forest import IsolationForestDetector
from src.models.deep_clustering import DeepClusteringDetector
from src.evaluation.metrics import compute_metrics
from src.evaluation.owasp_mapper import evaluate_owasp_detection

## Generate Synthetic OWASP Anomalies

In [ ]:
trail = load_trail_dataset()
mixed, labels, owasp_cats = generate_anomalous_traces(
    trail['traces'], anomaly_ratio=0.3, seed=42
)

ext = AgentTraceFeatureExtractor()
X, _, _ = ext.extract_batch(mixed)
X = UBFSNormalizer('zscore').fit_transform(X)

X_normal = X[labels == 0]
print(f'Total: {len(X)}, Normal: {len(X_normal)}, Anomalous: {labels.sum()}')
print(f'Categories: {set(c for c in owasp_cats if c)}')

## Detection Matrix

In [ ]:
# Load pre-computed results
with open('../results/tables/experiment_3_owasp.json') as f:
    results = json.load(f)

categories = ['ASI01', 'ASI02', 'ASI05', 'ASI09', 'ASI10']
cat_labels = ['ASI01\nGoal Hijack', 'ASI02\nTool Misuse', 'ASI05\nMemory\nPoisoning',
              'ASI09\nExcessive\nAgency', 'ASI10\nRogue\nAgents']
model_names = list(results.keys())

matrix = np.zeros((len(model_names), len(categories)))
for i, model in enumerate(model_names):
    for j, cat in enumerate(categories):
        per_cat = results[model]['per_category']
        if cat in per_cat and isinstance(per_cat[cat], dict):
            matrix[i, j] = per_cat[cat].get('auc_roc', 0)

fig, ax = plt.subplots(figsize=(8, 3))
sns.heatmap(matrix, annot=True, fmt='.3f', cmap='RdYlGn', vmin=0, vmax=1,
            xticklabels=cat_labels, yticklabels=model_names, ax=ax)
ax.set_ylabel('Model')
plt.tight_layout()
plt.show()

## Key Finding

**Tool Misuse (ASI02)** is a consistent blind spot across all models. It changes parameters without changing structural patterns. This is the AI equivalent of an employee using legitimate access for illegitimate purposes — structurally undetectable.